In [61]:
import pandas as pd
import numpy as np
import plotly.express as px
# pandas_ta is not built into google colab so must download
!pip install pandas_ta
import pandas_ta as ta
import matplotlib.pyplot as plt

!pip install arch 
from arch import arch_model


!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 903 kB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import requests
import pandas as pd
from datetime import datetime
import os
rest_api = 'https://ftx.com/api'
markets_endpoint = '/markets'
historical_price_endpoint = '/markets/{}/candles?resolution={}&start_time={}&end_time={}'
UNIX_START = pd.Timestamp(0, unit='s', tz='UTC')
MAX_RESOLUTION = 86400 * 30
API_RECORD_LIMIT = 1500
def get_unix_ts(date_str):
    return (pd.Timestamp(date_str, tz='UTC') - UNIX_START) // pd.Timedelta('1s')

def current_time():
    return get_unix_ts(datetime.utcnow())
def get_market_start(market_name):
    url = rest_api + historical_price_endpoint.format(market_name, MAX_RESOLUTION, 0, current_time())

    res = requests.get(url)
    if res.status_code != 200:
        raise f"Status Code is {res.status_code}, Something went wrong"

    prices = res.json()['result']
    if not len(prices):
        raise f"No price data was found"

    unix_ts = int(prices[0]['time'] // 1e3)
    return unix_ts
def get_all_market_prices(market_name, resolution):

    market_start = get_market_start(market_name)
    market_end = current_time() 

    df_list = []
    for start_ts in range(market_start, market_end, resolution * API_RECORD_LIMIT):
        end_ts = start_ts + resolution * (API_RECORD_LIMIT - 1)
        url = rest_api + historical_price_endpoint.format(market_name, resolution, start_ts, end_ts)
        res = requests.get(url)
        df_list.append(res.json()['result'])

    flattened_df_list = [item for sublist in df_list for item in sublist]
    df = pd.DataFrame(flattened_df_list)
    return df
res = requests.get(rest_api + markets_endpoint)
markets = pd.DataFrame(res.json()['result'])
top_10_markets = markets.iloc[markets.volumeUsd24h.nlargest(10).index].name.values
resolution = 900

for market in top_10_markets:
    print(f"Fetching Data for the {market} Market")
    df = get_all_market_prices(market, resolution)

    # ensure correct folders exist
    folder = '/'.join(f'data/{market}.csv'.split('/')[:-1])
    if not os.path.exists(folder):
        os.makedirs(folder)

    df.to_csv(f'data/{market}.csv')

Fetching Data for the ETH-PERP Market
Fetching Data for the BTC-PERP Market
Fetching Data for the ETH/USD Market
Fetching Data for the BTC/USD Market
Fetching Data for the ETC-PERP Market
Fetching Data for the ETH-0930 Market
Fetching Data for the SOL-PERP Market
Fetching Data for the ETH/USDT Market
Fetching Data for the ETH-1230 Market
Fetching Data for the ATOM-PERP Market


In [69]:
btc = pd.read_csv('data/BTC/USD.csv').head(10000)
eth = pd.read_csv('data/ETH/USD.csv').head(10000)

fig = px.scatter(btc, x='time', y='close')
fig.show()

fig2 = px.scatter(eth, x='time', y='close')
fig2.show()

In [76]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() 
X = np.array(btc['time']).reshape(-1, 1)
flatX = [item for sublist in X for item in sublist]

Y = np.array(btc['close']).reshape(-1, 1)
flatY = [item for sublist in Y for item in sublist]
 
model.fit(X, Y)
print(model.score(X, Y))

# Predicted 'close' value according to linear model
Xpredicted = model.predict(X)
# {predicted 'close'} - {actual 'close'}
epsilon = [(Y[i][0] - Xpredicted[i][0]) for i in range(len(X))]

# Plot {predicted 'close'} - {actual 'close'} against time
fig = px.line(x = flatX, y = epsilon)
fig.show()

train = [[flatX[i], flatY[i]] for i in range(len(flatX))]

a_model = arch_model(train, p = 1, q = 1 )


print(epsilon)

print(Y)

print(Xpredicted)


0.5270466276701062


TypeError: ignored